In [1]:
"""Jupyter Notebook to Analyze all the simulations"""


import sys
import numpy as np
import MDAnalysis as mda
import matplotlib.pyplot as plt
import os
import fnmatch
import yaml
import gc
import math
import warnings
import re

from optparse import OptionParser
from collections import OrderedDict
from operator import add
from linecache import getline

from datetime import date
today = date.today()

gc.collect()

0

In [382]:
folder_path="/media/nenciric/Ricky2020/simulations/"
systems=["dibucaine","etidocaine","SMS","TPP"]
#systems=["etidocaine","SMS","TPP"]
#systems=["dibucaine"]

In [53]:
#class by Joe Melcr
#upgraded by Anne
#from https://github.com/akiirik/Databank/blob/main/Scripts/BuildDatabank/OrderParameter.py
class OrderParameter:
    """
    Class for storing&manipulating
    order parameter (OP) related metadata (definition, name, ...)
    and OP trajectories
    and methods to evaluate OPs.
    """
    def __init__(self, name, resname, atom_A_name, atom_B_name, *args):  #removed name, resname from arguments
        """
        it doesn't matter which comes first,
        atom A or B, for OP calculation.
        """
        #        self.name = name             # name of the order parameter, a label
        self.resname = resname       # name of residue atoms are in
        self.atAname = atom_A_name
        self.atBname = atom_B_name

        
        self.name = name
        #M_atom_A_name + " " + M_atom_B_name # generic name of atom A and atom B
        for field in self.__dict__:
            if not isinstance(field, str):
                warnings.warn("provided name >> {} << is not a string! \n \
                Unexpected behaviour might occur.")#.format(field)
            else:
                if not field.strip():
                    raise RuntimeError("provided name >> {} << is empty! \n \
                    Cannot use empty names for atoms and OP definitions.")#.format(field)
        # extra optional arguments allow setting avg,std values -- suitable for reading-in results of this script
        if len(args) == 0:
            self.avg = None
            self.std = None
            self.stem = None
        elif len(args) == 2:
            self.avg = args[0]
            self.std = args[1]
            self.stem = None
        else:
            warnings.warn("Number of optional positional arguments is {len}, not 2 or 0. Args: {args}\nWrong file format?")
        self.traj = []  # for storing OPs
        self.selection = []
        

    def calc_OP(self, atoms):
        """
        calculates Order Parameter according to equation
        S = 1/2 * (3*cos(theta)^2 -1)
        """
        # print(atoms)
        bond_len_max=1.5  # in A, max distance between atoms for reasonable OP calculation
        bond_len_max_sq=bond_len_max**2
        
        vec = atoms[1].position - atoms[0].position
        d2 = np.square(vec).sum()

        if d2>bond_len_max_sq:
            at1=atoms[0].name
            at2=atoms[1].name
            resnr=atoms[0].resid
            d=math.sqrt(d2)
            warnings.warn("Atomic distance for atoms \
            {at1} and {at2} in residue no. {resnr} is suspiciously \
            long: {d}!\nPBC removed???")
        cos2 = vec[2]**2/d2
        S = 0.5*(3.0*cos2-1.0)
        return S
        


    @property
    def get_avg_std_OP(self):
        """
        Provides average and stddev of all OPs in self.traj
        """
        # convert to numpy array
        return (np.mean(self.traj), np.std(self.traj))
    @property
    def get_avg_std_stem_OP(self):
        """
        Provides average and stddev of all OPs in self.traj
        """
        std=np.std(self.traj)
        # convert to numpy array
        return (np.mean(self.traj),std,std/np.sqrt(len(self.traj)-1))
    @property
    def get_op_res(self):
        """
        Provides average and stddev of all OPs in self.traj
        """

        # convert to numpy array
        return self.traj

def go_through_simulation(path,name,system):
    topology = path+name+"/"+name+".gro"
    trajectory = path+name+"/"+name+".xtc"
    top_file = path+name+"/"+name+".top"
    topology_tpr = path+name+"/"+name+".tpr"
    readme = path+name+ "/README.yaml"
    today = str(date.today())
    
    u = mda.Universe(topology,trajectory)
    
    if not os.path.isfile(readme):
        sim={}
    else:
        with open(readme) as yaml_file:
            sim = yaml.load(yaml_file, Loader=yaml.FullLoader)
       
    begin_time=u.trajectory.time

    if not  'TRAJECTORY_SIZE' in sim:   
        sim['TRAJECTORY_SIZE'] = os.path.getsize(trajectory)/1048576

    Nframes=len(u.trajectory)
    timestep = u.trajectory.dt
    
    if not 'TIMESTEP' in sim: 
        sim['TIMESTEP'] = timestep
    
    trj_length = Nframes * timestep
    
    if not 'TRJLENGTH' in sim:
        sim['TRJLENGTH'] = trj_length
        
    if not 'TRJBEGIN' in sim:
        sim['TRJBEGIN'] = begin_time
        
    if not 'TRJEND' in sim:
        sim['TRJEND'] = begin_time+trj_length
        
    if not 'BINDINGEQ' in sim:
        sim['BINDINGEQ'] = input("Biding of {} eqilibrated after [ps] \n".format(output))

        
    if not 'TEMPERATURE' in sim:
        #get temperature from tpr; taken from AddData.py by Anne Kiirikki
        file1 =  'temporary_tpr.txt'

        print("Exporting information with gmx dump")  

        os.system('echo System | gmx dump -s '+ topology_tpr + ' > '+file1)

        with open(file1, 'rt') as tpr_info:
            for line in tpr_info:
                if 'ref-t' in line:
                    sim['TEMPERATURE']=float(line.split()[1])
  
    
    if not 'COMPOSITION' in sim:
        sim["COMPOSITION"]={}
        with open(top_file,"r") as f:
            molecules_list = False
            for line in f.readlines():
                if molecules_list:
                    if not line.startswith(";"):
                        items = line.split()
                        if len(items)==2:
                            sim["COMPOSITION"][items[0]]=items[1]
                elif line.startswith("[ molecules ]"):
                    molecules_list = True

        


    with open(readme, 'w') as f:
        yaml.dump(sim,f, sort_keys=False)
        
        
    with open(system +"_"+ today+ '.out', 'a') as f:
        try:
            f.write("{:80} {:>5} {:>6} {:>4} {:>4} ".format(name,int(timestep),sim['TRJLENGTH']/1000,int(int(sim['BINDINGEQ'])/1000),sim['TEMPERATURE']))
        except:
            f.write("{:80} {:>5} {:>6} {:>4} {:>4} ".format(name,int(timestep),sim['TRJLENGTH']/1000,sim['BINDINGEQ'],sim['TEMPERATURE']))

        for key in sim["COMPOSITION"]:
            f.write("{:>5}: {:>6}, ".format(key, sim["COMPOSITION"][key]))
        f.write("\n")
        
    print("great success!!!")
    
def initialize_output(system,folder_path):
    today = str(date.today())
    with open(system +"_"+ today+ '.out', 'w') as f:
        f.write("# List of simulations for: {} \n".format(system))
        f.write("# Path: {} \n".format(folder_path))
        f.write("# Create on: {} \n \n".format(today))
        f.write("#     1 - System \n")
        f.write("#     2 - Saving frequency [ps]  \n")
        f.write("#     3 - Total length [ns] \n")
        f.write("#     4 - Binding equilibrated after [ns]  \n")
        f.write("#     5 - Temperature [K] \n")
        f.write("#     6 - Composition \n \n")

def initialize_output_analyze(system,folder_path):
    """Makes header for the output file, hard-coded, needs to be developed"""
    today = str(date.today())
    column=5
    with open(localPath + system + '_analysis_'+ today +'.out', 'w') as f:
            f.write("# List of simulations for: {} \n".format(system))
            f.write("# Path: {} \n".format(folder_path))
            f.write("# Create on: {} \n \n".format(today))
            f.write("#     1 - System \n")
            f.write("#     2 - Start time [ns] \n")
            f.write("#     3 - End time [ns] \n")
            f.write("#     4 - Saving frequency [ps]  \n")
          
    ordPars = {}   
    with open(localPath+"OP_def.def","r") as f:
            for line in f.readlines():
                if not line.startswith("#"):
                    items = line.split()
                    ordPars[items[0]] = OrderParameter(*items)
    
    with open(localPath+ system + '_analysis_'+ today +'.out', 'a') as f:
        f.write("#     {}-{} - ".format(column,column+8))
        for op in ordPars:
            f.write("{} , error {}, ".format(op, op))
        f.write(" \n")
    column+=9
        
    with open(localPath + system + '_analysis_'+ today +'.out', 'a') as f:
        f.write("#     {}-{} - Box - Z,  APL \n".format(column,column+1))
    column+=2


            
            
"""Analysis of different stuff - to be developed"""
class AnalysisToolbox:
    def __init__(self,path,name,system,analysis):
        self.topology = path+name+"/"+name+".gro"
        self.topology_tpr = path+name+"/"+name+".tpr"
        self.trajectory = path+name+"/"+name+".xtc"
        self.mol = mda.Universe(self.topology,self.trajectory)
        print(self.mol)
        self.output = localPath + name
        self.today = str(date.today())

        
        readme = path+name+ "/README.yaml"
        with open(readme) as yaml_file:
            content = yaml.load(yaml_file, Loader=yaml.FullLoader)
        
        self.readme=content
        self.Nframes=len(self.mol.trajectory)-(int(self.readme['BINDINGEQ'])/int(self.readme['TIMESTEP']))
        
        self.system= localPath + system
        choose_function = {"box_dimensions": [self.ini_box_dimensions,self.box_dimensions,self.fin_box_dimensions],
                           "order_parameter": [self.ini_order_parameter,self.order_parameter,self.fin_order_parameter]}
        
        print(name)

        if not 'ANALYSIS' in self.readme:
            self.readme['ANALYSIS']={}

        
        self.column=5
        try:
            for analyze in analysis:
                choose_function[analyze][0]()

            
            with open(self.system + '_analysis_'+ self.today +'.out', 'a') as f:
                f.write(" \n")

            begin_analysis=int(int(self.readme['BINDINGEQ'])/int(self.readme['TIMESTEP']))
            print("going throught the trajectory")
            for self.frame in self.mol.trajectory[begin_analysis:]:
                last_frame=self.frame.time
                #print(last_frame)
                for analyze in analysis:
                    choose_function[analyze][1]()
                
            print("exiting trajectory")
            with open(self.system + '_analysis_'+ self.today +'.out', 'a') as f:
                f.write("{:75} {:>10} {:>10} {:>10} ".format(
                    name,int(self.readme['BINDINGEQ'])/1000,last_frame/1000,int(self.readme['TIMESTEP'])))
        
            for analyze in analysis:
                choose_function[analyze][2]()
        except Exception as e:
            print(e)
            print("some trouble")
            with open(self.system + '_analysis_'+ self.today +'.out', 'a') as f:
                f.write("{:75}  Trouble ".format(name))
            try:
                with open(self.system + '_analysis_'+ self.today +'.out', 'a') as f:
                    f.write("eqil time: {}, total time: {} \n".format(self.readme['BINDINGEQ'],self.readme['TRJLENGTH']))
            except:
                pass
            
            try:
                os.system('rm ' + self.output + '.xtc' ) 
            except:
                pass
        
             
        with open(readme, 'w') as f:
            yaml.dump(self.readme,f, sort_keys=False)
                    
        

    def ini_box_dimensions(self):
        """At the moment assumes 100 lipids per leaflet"""

        self.box_sizes=[]
        

        
        
    def ini_order_parameter(self):
        
        """
        parses input file with Order Parameter definitions
        file format is as follows:
        OP_name    resname    atom1    atom2  +extra: OP_mean  OP_std
        (flexible cols)
        fname : string
        input file name
        returns : dictionary
        with OrderParameters class instances
        """
        
        print("Make molecules whole in the trajectory")
        os.system('echo System | gmx trjconv -f ' + self.trajectory + ' -s ' + self.topology_tpr + ' -o ' + self.output + ' -pbc mol' )  # -b ' + str(EQtime))
        self.mol = mda.Universe(self.topology,self.output+'.xtc')
        self.Nframes=len(self.mol.trajectory)-(int(self.readme['BINDINGEQ'])/int(self.readme['TIMESTEP']))
        self.ordPars = {}

        with open(localPath+"OP_def.def","r") as f:
            for line in f.readlines():
                if not line.startswith("#"):
                    items = line.split()
                    self.ordPars[items[0]] = OrderParameter(*items)
        
        
        
    
        # make atom selections for each OP and store it as its attribute for later use in trajectory
        for self.op in self.ordPars.values():
            # selection = pairs of atoms, split-by residues
            selection = self.mol.select_atoms("resname {rnm} and name {atA} {atB}".format(
                                    rnm=self.op.resname, atA=self.op.atAname, atB=self.op.atBname)
                                    ).atoms.split("residue")

            for res in selection:
                # check if we have only 2 atoms (A & B) selected
                if res.n_atoms != 2:
                    for atom in res.atoms:
                        atA=self.op.atAname
                        atB=self.op.atBname
                        nat=res.n_atoms
                        warnings.warn("Selection >> name {atA} {atB} << \
                           contains {nat} atoms, but should contain exactly 2!")
            self.op.selection = selection

        
            self.Nres=len(self.op.selection)
            self.op.traj= [0]*self.Nres

    
    def box_dimensions(self):
        current_box_z = self.frame.dimensions[2]/10
        current_box_x = self.frame.dimensions[0]/10
          
        current_time = self.frame.time
           
        self.box_sizes.append([current_time,current_box_z,current_box_x])

        
        
    def order_parameter(self):
        for self.op in self.ordPars.values():     
            for i in range(0,self.Nres):
                residue=self.op.selection[i]
                S = self.op.calc_OP(residue)

                self.op.traj[i]=self.op.traj[i]+S/self.Nframes

                
    def fin_order_parameter(self):
        if not 'ORDER_PARAMETER' in self.readme['ANALYSIS']:
                self.readme['ANALYSIS']['ORDER_PARAMETER']={}
        with open(self.output+ "_order_parameter_" + str(self.today),"w") as f:
            f.write("Order parameter analysis \n")
            f.write("Analyzed from {} to {} ns. With saving frequency {} ps.".format(int(self.readme['BINDINGEQ'])/1000,int(self.readme['TRJLENGTH'])/1000,int(self.readme['TIMESTEP'])))
            f.write("# OP_name    resname    atom1    atom2    OP_mean   OP_stddev   OP_err.est. \n\
#--------------------------------------------------------------------------------------------\n" )
            for self.op in self.ordPars.values():
                (self.op.mean, self.op.std, self.op.stem) = self.op.get_avg_std_stem_OP
                f.write( "{:12s} {:10s} {:8s} {:7s} {: 2.5f}  {: 2.5f}    {: 2.5f} \n".format(
                         self.op.name, self.op.resname, self.op.atAname, self.op.atBname,
                         self.op.mean, self.op.std, self.op.stem)
                       )
                with open(self.system + '_analysis_'+ self.today +'.out', 'a') as g:
                    g.write("{: 2.5f} {: 2.5f}  ".format(
                        self.op.mean, self.op.stem))
                try:
                    self.readme['ANALYSIS']['ORDER_PARAMETER'][self.op.name]=float(self.op.mean)
                    self.readme['ANALYSIS']['ORDER_PARAMETER'][self.op.name+"_error"]=float(self.op.stem)
                except:
                    print("problem with writing OPs to readme")
        os.system('rm ' + self.output + '.xtc' ) 
        
        
        


    def fin_box_dimensions(self):
        average_dimensions=np.average(self.box_sizes,axis=0)
        with open(self.system + '_analysis_'+ self.today +'.out', 'a') as f:
            f.write("{: 3.2f} {: 3.2f}  ".format( average_dimensions[1], average_dimensions[2]**2/100))
        with open(self.output+'_boxSizes_' +self.today+ '.out', 'w') as f:
            np.savetxt(f, self.box_sizes,fmt='%8.4f  %.8f %.8f')
            
        if not 'BOX_DIMENSIONS' in self.readme['ANALYSIS']:
            self.readme['ANALYSIS']['BOX_DIMENSIONS']={}
        

        self.readme['ANALYSIS']['BOX_DIMENSIONS']['REPEAT_DISTANCE']=float(average_dimensions[1])
        self.readme['ANALYSIS']['BOX_DIMENSIONS']['AREA_PER_LIPID']=float(average_dimensions[2]**2/100) 
       

        
def box_dimensions(topology,trajectory,output,system):
    
    u = mda.Universe(topology,trajectory)
    box_sizes=[]
    
    """Go through the simulation - do I want this???, gets APL as a function of time"""
    for ts in u.trajectory:
        #count the index of the frame, numbered from 0, 
        frame = ts.frame  

            
        #reads the dimension (in Angstroms)
        current_box_z = ts.dimensions[2]
        current_box_x = ts.dimensions[0]
          
        last_time = ts.time

            
        box_sizes.append([last_time,current_box_x/10,current_box_z/10])

        
    with open(output+'_boxSizes.out', 'w') as f:
        np.savetxt(f, box_sizes,fmt='%8.4f  %.8f %.8f')

    

In [3]:
cd /media/ricky/One\ Touch/

/media/ricky/One Touch


In [10]:
folder_path="simulations/"
systems=["etidocaine"]
localPath='/home/ricky/Documents/from_work/charged_molecules_binding/figure3_SI/OP_check_for_errors_in_script/'

"""Go through all simulations and calculate OP and box dimentions"""
for file in os.listdir(folder_path):
    for system in systems:
        initialize_output_analyze(system,folder_path)

In [51]:
#Test the correctness of OP calculaions - 5.1.2022
eti_140=["etidocaine_POPC_CHARMM_298K_Cl_countra_140mM_5200waters","etidocaine_POPC_CHARMM_298K_Cl_countra_140mM_5985waters","etidocaine_POPC_CHARMM_298K_Cl_countra_140mM_7581waters","etidocaine_POPC_CHARMM_298K_Cl_countra_140mM_9975waters","etidocaine_POPC_CHARMM_298K_Cl_countra_140mM_20000waters_2746","etidocaine_POPC_CHARMM_298K_Cl_countra_140mM_177600waters_413"]
eti_280=["etidocaine_POPC_CHARMM_298K_Cl_countra_280mM_5200waters","etidocaine_POPC_CHARMM_298K_Cl_countra_280mM_19900waters_2785"]
eti_420=["etidocaine_POPC_CHARMM_298K_Cl_countra_420mM_5200waters","etidocaine_POPC_CHARMM_298K_Cl_countra_420mM_19900waters_1212"]
#file_name="etidocaine_POPC_CHARMM_298K_Cl_countra_140mM_5200waters"
for file_name in eti_140:
    AnalysisToolbox(folder_path,file_name,system,["order_parameter"])

<Universe with 43011 atoms>
etidocaine_POPC_CHARMM_298K_Cl_countra_140mM_5200waters
Make molecules whole in the trajectory
712000


/home/ricky/anaconda3/envs/all3/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:196: UserWarning: Reload offsets from trajectory
 ctime or size or n_atoms did not match
  warnings.warn("Reload offsets from trajectory\n "


going throught the trajectory
exiting trajectory
200
0.022482825664106018
[0.034817674507204625, 0.023829025741066923, 0.06855417706427641, -0.0007282841227519584, 0.011872138796603156, 0.013019942607579289, 0.04407073161829694, 0.03694168006929091, 0.012924944610059089, 0.017574860372094346, 0.06470924055894339, 0.011976679314280475, 0.03324017181257706, 0.05578931150219972, 0.034910478743049984, 0.056673829311449125, 0.03244093213580645, 0.020956641010758625, 0.03780795059035132, 0.041094687025442334, 0.03175049573622247, 0.03621914568048165, 0.034062182984399114, 0.012863002265188377, 0.031658006302125306, 0.01647833184288275, 0.05304459639675286, 0.071122560376476, 0.012849084577227245, 0.019938371339580315, 0.053888588662713274, 0.06893681301245486, 0.03663626080254473, 0.022185454736712844, 0.03908090435865808, 0.012889147789747434, 0.046426547194620055, 0.03945625990091938, 0.008038347157163561, 0.014615726271413061, 0.012745738265442093, 0.039076440190242846, 0.0244671126468293

<Universe with 45460 atoms>
etidocaine_POPC_CHARMM_298K_Cl_countra_140mM_5985waters
Make molecules whole in the trajectory
1122200
going throught the trajectory
exiting trajectory
200
0.020682937072717057
[0.005358237036952673, 0.02020683690492658, 0.04415440889063516, 0.06143996774835916, 0.022717525928909275, 0.026554185571156998, -0.0004599310653984646, 0.04578044421350149, 0.05035314373356777, 0.01656638082631052, 0.044121201139081814, 0.01623147356624441, 0.042867117678943405, 0.032975003070967616, 0.03229103677903496, -0.006891378290216689, 0.03758049260720968, 0.020365226108285193, 0.03663504178948337, 0.011910572955454755, 0.025446060690628765, 0.017173701764557896, 0.022773434275082494, 0.03620357741343318, 0.04962430342282259, -0.005746347238717397, 0.019991242406406044, 0.028058924315473934, 0.03452310036390759, 0.043687471498065955, 0.02144818186394546, 0.03762457393456982, -0.0021383061850890916, 0.029089831891470866, 0.012185193720683761, 0.037965205639304625, 0.049955894

<Universe with 50436 atoms>
etidocaine_POPC_CHARMM_298K_Cl_countra_140mM_7581waters
Make molecules whole in the trajectory
986600
going throught the trajectory
exiting trajectory
200
0.015104576029997182
[0.017852599945280982, 0.012321617203150963, 0.001549517150707789, -0.012948584800161898, 0.007421477318234579, 0.019855348409099558, 0.03813394427965444, -0.039763589391902505, -0.01098749935880962, 0.027259211677029754, 0.023958930177877504, -0.0026436403898402764, 0.00946421763541297, 0.012246311464963947, 0.01749593144585692, 0.016876291385963776, 0.03365678860664648, 0.026990139310928935, 0.03841391231142305, 0.015046938328434593, -0.013483388264718808, 0.022192449836794276, -0.0197768100954124, 0.024872355982451924, 0.012326107373874325, 0.030889969312978372, 0.024710865995769674, -0.026305806480558757, -0.018103759171102112, 0.04033715102099928, 0.029021729138009836, 0.02680108988861864, 0.02031541987577251, 0.04160469802996841, 0.05787910549846203, 0.02587930101017154, 0.016837

<Universe with 57900 atoms>
etidocaine_POPC_CHARMM_298K_Cl_countra_140mM_9975waters
Make molecules whole in the trajectory
1753400
going throught the trajectory
exiting trajectory
200
0.012062931296429609
[0.007535143142550262, 0.013281843276406326, 0.0019686832895366452, 0.01808643374671485, 0.004597192251944023, 0.008816183675657728, 0.01150300227980908, 0.004438911744276301, 0.028206396126954972, 0.011271944357514045, 0.023689858210673393, -0.012977989456976488, 0.016215655938222277, 0.02923025644573317, 0.008563601518482457, 0.00041899743957951374, 0.01724971648312601, 0.02252352574970241, 0.008978651839607409, 0.004209175292215564, 0.020878075784786343, -0.0013176408333221174, 0.021206344967517974, 0.006592850865462961, 0.01286099620176247, 0.025674808960201672, 0.017568644101302416, 0.015886264234175492, 0.01402170746624726, 0.020684671590911013, 0.004244227246460603, 0.019146027917330527, 0.004815730561862041, 0.02395641756397466, 0.02533230714699652, -0.004638831786958263, 0.02

<Universe with 89300 atoms>
etidocaine_POPC_CHARMM_298K_Cl_countra_140mM_20000waters_2746
Make molecules whole in the trajectory
1999100
going throught the trajectory
exiting trajectory
200
0.006291230552318881
[0.0216106119753874, -0.01781886090482255, -0.006612486572977473, 0.004509351196805904, 0.029818447046835717, 0.018999406008569555, 0.008230780245745455, 0.021542700038940214, -0.012383927175986902, 0.014094353342666928, -0.008472561418702836, 0.009095113189301585, -0.0002510694977201032, 7.759348936156378e-07, -0.0012581435356904943, -0.0026199133607447763, 0.020509660535845912, 0.014057801149451493, 0.02992507103936107, 0.002173789957621097, -0.012265033637944027, 0.014043311671808103, 0.01590805569198271, 0.011406151882549832, 0.014280738173302773, 0.01750867812600384, 0.005655009494497234, -0.0050628775072016656, 0.0005567600375984986, 0.021230102644797126, 0.013934326135333056, -0.0018431674708834689, -0.008288934136643868, 0.017284243226941366, 0.008913100926182482, -0.022

<Universe with 581800 atoms>
etidocaine_POPC_CHARMM_298K_Cl_countra_140mM_177600waters_413
Make molecules whole in the trajectory
363000
going throught the trajectory
exiting trajectory
200
-0.0038965607049216045
[-0.03420368392152707, -0.005252092102419046, 0.03031521940875311, -0.011875009795871796, 0.003919493427626951, 0.033900479739416026, -0.0002803907640919743, 0.052711910200979366, 0.014138230243265618, 0.010159190073743869, -0.05983943946067059, 0.0222229120372834, -0.012535470662316483, -0.019909629278889042, 0.026953573158761915, 0.050498508639347926, 0.011532829297868745, -0.01492568135736224, -0.05010262936560066, -0.01594758965612044, -0.0424837563987333, 0.01353183257848992, -0.058264753912406306, 0.03149313731232857, -0.03142149237940533, 0.042420927726072984, -0.023911256981488275, 0.027247335107152017, -0.0035225186879458683, 0.004125960585323227, -0.028280943863933716, -0.03619895361077745, -0.012467285647893565, -0.0006213402238825738, 0.04400619873101107, 0.0258725

In [54]:
eti_280=["etidocaine_POPC_CHARMM_298K_Cl_countra_280mM_5200waters","etidocaine_POPC_CHARMM_298K_Cl_countra_280mM_19900waters_2785"]
eti_420=["etidocaine_POPC_CHARMM_298K_Cl_countra_420mM_5200waters","etidocaine_POPC_CHARMM_298K_Cl_countra_420mM_19900waters_1212"]
for file_name in eti_280:
    AnalysisToolbox(folder_path,file_name,system,["order_parameter"])
    
for file_name in eti_420:
    AnalysisToolbox(folder_path,file_name,system,["order_parameter"])

<Universe with 43622 atoms>
etidocaine_POPC_CHARMM_298K_Cl_countra_280mM_5200waters
Make molecules whole in the trajectory
going throught the trajectory
exiting trajectory
<Universe with 91200 atoms>
etidocaine_POPC_CHARMM_298K_Cl_countra_280mM_19900waters_2785
Make molecules whole in the trajectory
going throught the trajectory
exiting trajectory
<Universe with 44350 atoms>
etidocaine_POPC_CHARMM_298K_Cl_countra_420mM_5200waters
Make molecules whole in the trajectory
going throught the trajectory
exiting trajectory
<Universe with 93916 atoms>
etidocaine_POPC_CHARMM_298K_Cl_countra_420mM_19900waters_1212
Make molecules whole in the trajectory
going throught the trajectory
exiting trajectory


In [428]:
names=["TPP_POPC_CHARMM36_298K_280mM_20000waters_paramchem_B3LYP_ESP_charges"]
for name in names:
    AnalysisToolbox(folder_path,name,"TPP",["box_dimensions"])

TPP_POPC_CHARMM36_298K_280mM_20000waters_paramchem_B3LYP_ESP_charges
[]
going throught the trajectory
exiting trajectory
[2000000.0, 12.263045501708984, 8.559166717529298]
[400000.0, 12.334797668457032, 8.53302459716797]
[1.20000000e+06 1.20297174e+01 8.64676695e+00]


In [304]:
names=["TPP_POPC_CHARMM36_298K_280mM_20000waters_paramchem_B3LYP_ESP_charges","TPP_POPC_CHARMM36_298K_280mM_20000waters_paramchem_B3LYP_ESP_charges_ECC"]
for name in names:
    topology=folder_path+name+"/"+name+".gro"
    trajectory=folder_path+name+"/"+name+".xtc"
    topology_tpr=folder_path+name+"/"+name+".tpr"
    readme=folder_path+name+"/README.yaml"
    with open(readme) as yaml_file:
        content = yaml.load(yaml_file, Loader=yaml.FullLoader)
    AnalysisToolbox(topology,trajectory,name,system,["order_parameter","box_dimensions"],content,topology_tpr)

Make molecules whole in the trajectory
Make molecules whole in the trajectory


In [379]:
for system in systems:
    initialize_output(system,folder_path)
"""Go through all simulations and gather INFO on covergence"""
for file in os.listdir(folder_path):
    input_corr_file = folder_path+os.fsdecode(file)
    for system in systems:
        if fnmatch.fnmatch(os.fsdecode(file), "*"+system+"*"):
            print(os.fsdecode(file))

        
            go_through_simulation(folder_path,os.fsdecode(file),system)
 
       

dibucaine_POPC_CHARMM_298K_Cl_countra_104mM_8800waters
great success!!!
dibucaine_POPC_CHARMM_298K_Cl_countra_123mM
great success!!!
dibucaine_POPC_CHARMM_298K_Cl_countra_130mM_8800waters
great success!!!
dibucaine_POPC_CHARMM_298K_Cl_countra_13mM_8800waters
great success!!!
dibucaine_POPC_CHARMM_298K_Cl_countra_156mM_8800waters
great success!!!
dibucaine_POPC_CHARMM_298K_Cl_countra_15mM_177600waters
great success!!!
dibucaine_POPC_CHARMM_298K_Cl_countra_182mM_8800waters
great success!!!
dibucaine_POPC_CHARMM_298K_Cl_countra_208mM_8800waters
great success!!!
dibucaine_POPC_CHARMM_298K_Cl_countra_234mM_8800waters
great success!!!
dibucaine_POPC_CHARMM_298K_Cl_countra_260mM_8800waters
great success!!!
dibucaine_POPC_CHARMM_298K_Cl_countra_26mM_8800waters
great success!!!
dibucaine_POPC_CHARMM_298K_Cl_countra_286mM_8800waters
great success!!!
dibucaine_POPC_CHARMM_298K_Cl_countra_312mM_8800waters
great success!!!
dibucaine_POPC_CHARMM_298K_Cl_countra_52mM_8800waters
great success!!!
dibuc